In [1]:
from selenium import webdriver as wd
import requests

from bs4 import BeautifulSoup as bs

import pandas as pd
from urllib.parse import urlparse
import math
import csv
import time
import re

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
so_url = [
"https://shopee.vn/S%E1%BB%91t-Ch%E1%BA%A5m-L%E1%BA%A9u-Haidilao-S%E1%BB%91t-Ch%E1%BA%A5m-%C4%90%E1%BA%ADu-Ph%E1%BB%99ng-Ch%E1%BA%A5m-L%E1%BA%A9u-Tuy%E1%BB%87t-%C4%90%E1%BB%89nh-Haidilao-i.48794629.7631708571",
"https://shopee.vn/%E1%BB%9At-ch%C6%B0ng-D%E1%BA%A7u-Laoganma-%C4%90%E1%BB%A7-V%E1%BB%8B-%E2%9A%A1%E2%9A%A1Freeship%E2%9A%A1%E2%9A%A1-n%E1%BB%99i-%C4%91%E1%BB%8Ba-TQ-i.275350037.5761162273"
]

In [3]:
# 제품명과 리뷰수 추출 힘수
review_counts = []
product_names = []
def URL_1(url):
    driver =  wd.Chrome( r"chromedriver_win32\chromedriver.exe")
    driver.get(url)
    WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/div[2]/div[3]/div/div[1]/span'))
                )
    
    source = driver.page_source
    soup = bs(source, 'html.parser')
    review_count = soup.findAll('div', attrs = {'class':'OitLRu'})[1].get_text()
    review_counts.append(review_count)
    print(review_count)
    
    product_name = soup.findAll('div',{'class':'attM6y'})[0].get_text().replace('Yêu thích', ' ').strip()
    product_names.append(product_name)
    print(product_name)
    
    driver.close()
    driver.quit()

In [4]:
for i in so_url:
    URL_1(i)

783
+Sốt Chấm Lẩu Haidilao - Sốt Chấm Đậu Phộng Chấm Lẩu Tuyệt Đỉnh Haidilao
362
+Ớt chưng Dầu Laoganma Đủ Vị ⚡⚡Freeship⚡⚡ nội địa TQ


In [5]:
# json 형태로 데이터를 추출해야함
# 기존 url에서 itemid와 shopid 추출해서 json url 생성
json_urls = []
def json_URL(url):
    itemid = url.split('.')[3]
    shopid = url.split('.')[2]
    json_url = "https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid={}&limit=6&offset={}&shopid={}&type=0".format(itemid, "{}",shopid)
    json_urls.append(json_url)
    return json_urls

In [7]:
json_urls = []
for i in so_url:
    json_URL(i)
    print(json_urls)

['https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid=7631708571&limit=6&offset={}&shopid=48794629&type=0']
['https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid=7631708571&limit=6&offset={}&shopid=48794629&type=0', 'https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid=5761162273&limit=6&offset={}&shopid=275350037&type=0']


In [8]:
# 리뷰 수에 따른 json url 페이지 수
result_review = []
for i in range(0,len(review_counts)):
    Counts = review_counts[i]
    review_count = int(Counts.replace("k","00").replace(",",""))
    review_count = math.floor(review_count/6)
    review_count += 17
    result_review.append(review_count)
    print(result_review)

[147]
[147, 77]


In [9]:
# offset 생성 0,6,12 ...로 증가
total_urls=[]
def shopee(review_count, URL):
    for j in range(0,review_count+1):
        total_url = URL.format(6*j)
        total_urls.append(total_url)
        #print(total_url)

In [10]:
for i in range(0, len(json_urls)):
    shopee(result_review[i], json_urls[i])

In [11]:
total_urls

['https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid=7631708571&limit=6&offset=0&shopid=48794629&type=0',
 'https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid=7631708571&limit=6&offset=6&shopid=48794629&type=0',
 'https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid=7631708571&limit=6&offset=12&shopid=48794629&type=0',
 'https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid=7631708571&limit=6&offset=18&shopid=48794629&type=0',
 'https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid=7631708571&limit=6&offset=24&shopid=48794629&type=0',
 'https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid=7631708571&limit=6&offset=30&shopid=48794629&type=0',
 'https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid=7631708571&limit=6&offset=36&shopid=48794629&type=0',
 'https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid=7631708571&limit=6&offset=42&shopid=48794629&type=0',
 'https://shopee.vn/api/v2/item/ge

In [15]:
def shope_result(URL):    
    searchList = []

    r = requests.get(URL)
    json_data = r.json()
    try:
        for i in range(0,len(json_data['data']['ratings'])):

            temp = []
            #제품명
            product_name = json_data['data']['ratings'][0]['product_items'][0]['name']
            temp.append(product_name)

            # 작성자
            member_ID = json_data['data']['ratings'][i]['author_username']
            temp.append(member_ID)

            # 리뷰
            review = json_data['data']['ratings'][i]['comment']
            temp.append(review)

            try:
                searchList.append(temp)
                print("-----------------------------------------------------")
                print(product_name + ':' + member_ID + ':' + review)

            except:
                print("data가 없음")
    except:
        print('review가 없음 ')
            
    return searchList   


In [16]:
csvfile = open('./shopee.csv', 'w', encoding='utf-8-sig', newline= '')
csvwriter = csv.writer(csvfile)
for i in range(0,len(total_urls)):
    result = shope_result(total_urls[i])
    for row in result:
        csvwriter.writerow(row)

-----------------------------------------------------
[Mã 157FMCGSALE giảm 8% đơn 500K] Sốt Chấm Lẩu Haidilao - Sốt Chấm Đậu Phộng Chấm Lẩu Tuyệt Đỉnh Haidilao:d*****7:Chất lượng sản phẩm tốt, đóng gói sản phẩm rất đẹp và chắc chắn ......🥰🥰🥰🥰🥰🥰
Dịch nên giao hàng hơi lâu xíu mà có lẩu ăn là vui ròiii
-----------------------------------------------------
[Mã 157FMCGSALE giảm 8% đơn 500K] Sốt Chấm Lẩu Haidilao - Sốt Chấm Đậu Phộng Chấm Lẩu Tuyệt Đỉnh Haidilao:j*****4:Ydudifidcjdu. Jcic iduc k kckciduckb.  Đicjcjgk kvjc. Cụcc
-----------------------------------------------------
[Mã 157FMCGSALE giảm 8% đơn 500K] Sốt Chấm Lẩu Haidilao - Sốt Chấm Đậu Phộng Chấm Lẩu Tuyệt Đỉnh Haidilao:trangtlh:Tiện lợi vô cùng,mới thử gói đậu hủ tứ xuyên,vị hơi cứng ,nên thêm chút đường là ok ạ
-----------------------------------------------------
[Mã 157FMCGSALE giảm 8% đơn 500K] Sốt Chấm Lẩu Haidilao - Sốt Chấm Đậu Phộng Chấm Lẩu Tuyệt Đỉnh Haidilao:pinkerbellnguyen1604:Ship nhanh ạ. Cốt lẩu cay ăn khá th